<a href="https://colab.research.google.com/github/Nivedha-M/NIVEDHA/blob/master/Pyspark_data_eng_qns_ans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('creating_df').getOrCreate()
data = [(1,'Nive','ECE',4000),
	(2,'vishnu','ECE',8000),
	(3,'shweta','IT',12000)
	]

schema = ["ID","Name","DPT","Fees"]
spark = spark.builder.appName('creating_df').getOrCreate()
df = spark.createDataFrame(data=data,schema=schema)
group = df.groupBy(col('DPT')).agg(min(df.Fees).alias('Minimum_fees'),sum(df.Fees).alias('Total_fees'),count(df.DPT).alias('dpt_cnt'))
group.show()

+---+------------+----------+-------+
|DPT|Minimum_fees|Total_fees|dpt_cnt|
+---+------------+----------+-------+
|ECE|        4000|     12000|      2|
| IT|       12000|     12000|      1|
+---+------------+----------+-------+



In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from datetime import date

data = [(1,"Nivi",25),
	(2,"vishnu",24),
	(3,"shweta",26),
	(4,"Priya",28),
	(5,"Jambu",29)]

cols = ["ID","Name","age"]

df = spark.createDataFrame(data = data,schema = cols)
current_yr = date.today().year
usd_func = F.udf(lambda age:str(current_yr-age),T.StringType())
new = df.withColumn('Birth_year',usd_func(df.age))
new.show()

new.select('ID','Birth_year').show()

+---+------+---+----------+
| ID|  Name|age|Birth_year|
+---+------+---+----------+
|  1|  Nivi| 25|      1999|
|  2|vishnu| 24|      2000|
|  3|shweta| 26|      1998|
|  4| Priya| 28|      1996|
|  5| Jambu| 29|      1995|
+---+------+---+----------+

+---+----------+
| ID|Birth_year|
+---+----------+
|  1|      1999|
|  2|      2000|
|  3|      1998|
|  4|      1996|
|  5|      1995|
+---+----------+



In [ ]:
cols = ['Customer_id' , 'Account_id', 'Account_type']
table = [    ["Cust_id_1", "accnt_id_1", "loan_account"],
    ["Cust_id_1", "accnt_id_2", "savings_account"],
    ["Cust_id_1", "accnt_id_3", "demat_account"],
    ["Cust_id_2", "accnt_id_4", "loan_account"],
    ["Cust_id_2", "accnt_id_5", "loan_account"],
    ["Cust_id_2", "accnt_id_6", "savings_account"],
    ["Cust_id_3", "accnt_id_7", "demat_account"]
]

df = spark.createDataFrame(data=table,schema=cols)
# df.show()

# +----------+-----------------+
# |Customer_id|has_demat_account|
# +----------+-----------------+
# | Cust_id_1|                1|
# | Cust_id_2|                0|
# | Cust_id_3|                1|
# +----------+-----------------+

hi = df.groupBy('customer_id').agg(F.max(F.when(df.Account_type=='demat_account',1).otherwise(0)).alias('has_demat_account')).orderBy('Customer_id')
hi.show()




+-----------+----------+---------------+
|Customer_id|Account_id|   Account_type|
+-----------+----------+---------------+
|  Cust_id_1|accnt_id_1|   loan_account|
|  Cust_id_1|accnt_id_2|savings_account|
|  Cust_id_1|accnt_id_3|  demat_account|
|  Cust_id_2|accnt_id_4|   loan_account|
|  Cust_id_2|accnt_id_5|   loan_account|
|  Cust_id_2|accnt_id_6|savings_account|
|  Cust_id_3|accnt_id_7|  demat_account|
+-----------+----------+---------------+

+-----------+-----------------+
|customer_id|has_demat_account|
+-----------+-----------------+
|  Cust_id_1|                1|
|  Cust_id_2|                0|
|  Cust_id_3|                1|
+-----------+-----------------+



In [ ]:
from pyspark.sql.functions import split,explode
data = [('Alice','Badmiton,Tennis'),('Bob','Tennis,Cricket'),('Julie','Cricket,Carroms')]
cols = ['name','Hobbies']

df = spark.createDataFrame(data=data,schema=cols)
# df.show()
new = df.select('name',explode(split('Hobbies',',')).alias('exploded'))
# new = df.explode(split('Hobbies',','))
new.show()

+-----+--------+
| name|exploded|
+-----+--------+
|Alice|Badmiton|
|Alice|  Tennis|
|  Bob|  Tennis|
|  Bob| Cricket|
|Julie| Cricket|
|Julie| Carroms|
+-----+--------+



In [ ]:
data = [('a','aa',1),
	('a','aa',2),
	('b','bb',5),
	('b','bb',3),
	('b','bb',4)]
cols = ['col1','col2','col3']
df = spark.createDataFrame(data=data,schema=cols)
# df.show()
df1 = df.groupBy('col1','col2').agg(collect_list('col3').alias('col3'))
df1.show()

print("using sql query")

df.createOrReplaceTempView('my_temp_view')
df_sql = spark.sql('''
    select col1,col2, collect_list(col3) as col3 from my_temp_view group By col1,col2
    ''')
df_sql.show()

+----+----+---------+
|col1|col2|     col3|
+----+----+---------+
|   a|  aa|   [1, 2]|
|   b|  bb|[5, 3, 4]|
+----+----+---------+

using sql query
+----+----+---------+
|col1|col2|     col3|
+----+----+---------+
|   a|  aa|   [1, 2]|
|   b|  bb|[5, 3, 4]|
+----+----+---------+



In [ ]:
data = [{'dpt_id':'101','e_id':[10101,10102,10103]},
{'dpt_id':'102','e_id':[10201,10202]}]

df = spark.createDataFrame(data=data)
df.show()

df1 = df.select('dpt_id',explode('e_id').alias('e_id'))
df1.show()


+------+--------------------+
|dpt_id|                e_id|
+------+--------------------+
|   101|[10101, 10102, 10...|
|   102|      [10201, 10202]|
+------+--------------------+

+------+-----+
|dpt_id| e_id|
+------+-----+
|   101|10101|
|   101|10102|
|   101|10103|
|   102|10201|
|   102|10202|
+------+-----+



In [ ]:
# 7)create df in pyspark.Find avg stock value on daily basis of each stock.Find max avg stock value of each stock.

data = [('2023-01-01','AAPL',150.00),('2023-01-02','AAPL',155.00),
        ('2023-01-01','AAPL',200.00),('2023-01-01','MSFT',400.00),
	('2023-01-01','GOOG',2500.00),('2023-01-02','GOOG',2550.00),
	('2023-01-01','MSFT',300.00),('2023-01-02','MSFT',310.00)]
cols = ['date','stock','value']

df = spark.createDataFrame(data=data,schema=cols)
df = df.withColumn('date',to_date('date'))
avg_daily_basis = df.groupBy('date','stock').agg(avg('value').alias('avg_value'))
max_avg_each_stock = avg_daily_basis.groupBy('stock').agg(max('avg_value').alias('max_avg_value'))
max_avg_each_stock.show()


+-----+-------------+
|stock|max_avg_value|
+-----+-------------+
| AAPL|        175.0|
| GOOG|       2550.0|
| MSFT|        350.0|
+-----+-------------+



In [ ]:
table1 = [(1,'steve'),(2,'David'),(3,'Aryan')]
schema1 = ['student_id','student_name']

table2 = [(1,'pyspark','90'),(1,'sql',100),(2,'sql',70),(2,'pyspark',60),(3,'sql',30),(2,'pyspark',20)]
schema2 = ['student_id','subject_name','marks']


#OUTPUT:
# student_id	student_name	percentage	grade
# 1		Steve		95		Distinction
# 2		David		50		Second Class
# 3		Aryan		30		Fail

df1 = spark.createDataFrame(data=table1,schema=schema1)
df2 = spark.createDataFrame(data=table2,schema=schema2)

joined_df = df1.join(df2,df1.student_id==df2.student_id).drop(df2.student_id)
# joined_df.show()
new = joined_df.groupby('student_id','student_name').agg(avg('marks').alias('percentage'))
added_grade = new.select('*',when(new.percentage>=70,'Distinction')
                             .when((new.percentage<70)&(new.percentage>=60),'First Class')
                             .when((new.percentage<60)&(new.percentage>=50),'Second Class')
                             .when((new.percentage<50)&(new.percentage>=40),'Third Class')
                             .when(new.percentage<40,'Fail').alias('grade')
                             )
added_grade.show()


+----------+------------+----------+------------+
|student_id|student_name|percentage|       grade|
+----------+------------+----------+------------+
|         1|       steve|      95.0| Distinction|
|         2|       David|      50.0|Second Class|
|         3|       Aryan|      30.0|        Fail|
+----------+------------+----------+------------+



In [ ]:
# Remove the delimiters in a file pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col

spark = SparkSession.builder.appName('Data_Engineering_prbmls').getOrCreate()

# Sample data
data = ["1,Alice\t30|New York"]

df = spark.createDataFrame(data,"string")
splitted_value = split(df['value'],',|\t|\|')
df = df.withColumn('id',splitted_value.getItem(0))\
       .withColumn('Name',splitted_value.getItem(1))\
       .withColumn('age',splitted_value.getItem(2))\
       .withColumn('Country',splitted_value.getItem(3))
df.select('id','Name','age','Country').show()
df.show()

+---+-----+---+--------+
| id| Name|age| Country|
+---+-----+---+--------+
|  1|Alice| 30|New York|
+---+-----+---+--------+

+--------------------+---+-----+---+--------+
|               value| id| Name|age| Country|
+--------------------+---+-----+---+--------+
|1,Alice\t30|New York|  1|Alice| 30|New York|
+--------------------+---+-----+---+--------+



In [ ]:
#MISSING VALUE FROM THE LIST
from pyspark.sql.functions import col
data = [(1,), (2,), (4,), (5,), (7,), (8,), (10,),(11,)]
df = spark.createDataFrame(data=data,schema = ["Numbers"])
# df.show()
range_num = spark.range(1,11).toDF("Numbers")
# range_num.show()
new = range_num.join(df,"Numbers","left")
new.show()


+-------+
|Numbers|
+-------+
|      5|
|      1|
|      3|
|      2|
|      4|
|      7|
|      6|
|      9|
|     10|
|      8|
+-------+



In [ ]:
from pyspark.sql.functions import sum,count
from pyspark.sql.window import WindowSpec
data_movies = [(1, "Movie A"), (2, "Movie B"), (3, "Movie C"), (4, "Movie D"), (5, "Movie E")]

data_ratings = [(1, 101, 4.5), (1, 102, 4.0), (2, 103, 5.0),
                (2, 104, 3.5), (3, 105, 4.0), (3, 106, 4.0),
                (4, 107, 3.0), (5, 108, 2.5), (5, 109, 3.0)]

columns_movies = ["MovieID", "MovieName"]
columns_ratings = ["MovieID", "UserID", "Rating"]

df1 = spark.createDataFrame(data=data_movies,schema=columns_movies)
df2 = spark.createDataFrame(data=data_ratings,schema=columns_ratings)

avg_cnt_ratings = df2.groupBy('MovieID').agg((sum(col('Rating'))/count(col('Rating'))).alias('Avg_rating'))
top_3_rating = avg_cnt_ratings.orderBy(col("Avg_rating"),ascending=False).limit(3)
new = df1.join(top_3_rating,df1.MovieID==top_3_rating.MovieID,'inner').drop(top_3_rating.MovieID)
new.show()


+-------+---------+----------+
|MovieID|MovieName|Avg_rating|
+-------+---------+----------+
|      1|  Movie A|      4.25|
|      2|  Movie B|      4.25|
|      3|  Movie C|       4.0|
+-------+---------+----------+



In [ ]:
from pyspark.sql.functions import col,avg
data_movies = [(1, "Movie A"), (2, "Movie B"), (3, "Movie C"), (4, "Movie D"), (5, "Movie E")]

data_ratings = [(1, 101, 4.5), (1, 102, 4.0), (2, 103, 5.0),
                (2, 104, 3.5), (3, 105, 4.0), (3, 106, 4.0),
                (4, 107, 3.0), (5, 108, 2.5), (5, 109, 3.0)]

columns_movies = ["MovieID", "MovieName"]
columns_ratings = ["MovieID", "UserID", "Rating"]

df1 = spark.createDataFrame(data = data_movies,schema=columns_movies)
df2 = spark.createDataFrame(data = data_ratings,schema=columns_ratings)

avg_rating = df2.groupBy(col('MovieID')).agg((avg('Rating')).alias('Avg_rating'))
joined = df1.join(avg_rating,'MovieID','inner').orderBy("Avg_rating",ascending=False).limit(3)
joined.select("MovieID","MovieName").show()

+-------+---------+
|MovieID|MovieName|
+-------+---------+
|      1|  Movie A|
|      2|  Movie B|
|      3|  Movie C|
+-------+---------+



In [ ]:
# Sample data
from pyspark.sql import Row,SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
data = [Row(Date='2023-01-01', ProductID=100, QuantitySold=10),
        Row(Date='2023-01-02', ProductID=100, QuantitySold=15),
        Row(Date='2023-01-03', ProductID=100, QuantitySold=20),
        Row(Date='2023-01-04', ProductID=100, QuantitySold=25),
        Row(Date='2023-01-05', ProductID=100, QuantitySold=30),
        Row(Date='2023-01-06', ProductID=100, QuantitySold=35),
        Row(Date='2023-01-07', ProductID=100, QuantitySold=40),
        Row(Date='2023-01-08', ProductID=100, QuantitySold=45)]

df = spark.createDataFrame(data)
df = df.withColumn("Date",F.to_date(df.Date))
windowSpec = Window.partitionBy('ProductID').orderBy('Date').rowsBetween(-6,0)
df = df.withColumn('7 days Avg',F.avg('QuantitySold').over(windowSpec))
df.show()

+----------+---------+------------+----------+
|      Date|ProductID|QuantitySold|7 days Avg|
+----------+---------+------------+----------+
|2023-01-01|      100|          10|      10.0|
|2023-01-02|      100|          15|      12.5|
|2023-01-03|      100|          20|      15.0|
|2023-01-04|      100|          25|      17.5|
|2023-01-05|      100|          30|      20.0|
|2023-01-06|      100|          35|      22.5|
|2023-01-07|      100|          40|      25.0|
|2023-01-08|      100|          45|      30.0|
+----------+---------+------------+----------+



In [ ]:
from pyspark.sql import Row,SparkSession
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
data = [Row(UserID=4001, Age=17),
        Row(UserID=4002, Age=45),
        Row(UserID=4003, Age=65),
        Row(UserID=4004, Age=30),
        Row(UserID=4005, Age=80)]

df = spark.createDataFrame(data)

def age_group(age):
  if age<=17:
    return "Youth"
  elif age>17 and age<=45:
    return "Adult"
  else:
    return "senior"

df = spark.createDataFrame(data)
#F.udf(my_udf, returnType=DataType()) - syntax of udf
func_obj_udf = F.udf(age_group,StringType())
df = df.withColumn('Age_group',func_obj_udf('age'))
df.show()

+------+---+---------+
|UserID|Age|Age_group|
+------+---+---------+
|  4001| 17|    Youth|
|  4002| 45|    Adult|
|  4003| 65|   senior|
|  4004| 30|    Adult|
|  4005| 80|   senior|
+------+---+---------+



In [ ]:
#GET UNIQUE VISITORS
import pyspark.sql.functions as F
from pyspark.sql.functions import count_distinct
visitor_data = [Row(Date='2023-01-01', VisitorID=101),
                Row(Date='2023-01-01', VisitorID=102),
                Row(Date='2023-01-01', VisitorID=101),
                Row(Date='2023-01-02', VisitorID=103),
                Row(Date='2023-01-02', VisitorID=101)]

df = spark.createDataFrame(visitor_data)
df = df.withColumn("Date",F.to_date("Date"))
df = df.groupBy('Date').agg(count_distinct('VisitorID').alias('unique_visitors'))
df.show()

+----------+---------------+
|      Date|unique_visitors|
+----------+---------------+
|2023-01-01|              2|
|2023-01-02|              2|
+----------+---------------+



In [ ]:
#FIRST PURCHASE DATE OF EACH USER
from pyspark.sql.functions import *
purchase_data = [
    Row(UserID=1, PurchaseDate='2023-01-05'),
    Row(UserID=1, PurchaseDate='2023-01-10'),
    Row(UserID=2, PurchaseDate='2023-01-03'),
    Row(UserID=3, PurchaseDate='2023-01-12')
]

df = spark.createDataFrame(purchase_data)
df = df.withColumn("PurchaseDate",to_date("PurchaseDate"))
df = df.groupBy("UserID").agg(min('PurchaseDate').alias("first_purchase_date")).orderBy('UserID')
df.show()

+------+-------------------+
|UserID|first_purchase_date|
+------+-------------------+
|     1|         2023-01-05|
|     2|         2023-01-03|
|     3|         2023-01-12|
+------+-------------------+



In [ ]:
#Generate Sequential order of each row in each group
from pyspark.sql.functions import *
from pyspark.sql.window import Window
group_data = [
    Row(GroupID='A', Date='2023-01-01'),
    Row(GroupID='A', Date='2023-01-02'),
    Row(GroupID='B', Date='2023-01-01'),
    Row(GroupID='B', Date='2023-01-03')
]
df = spark.createDataFrame(group_data)
df = df.withColumn('Date',to_date('Date'))
windowspec = Window.partitionBy('GroupID').orderBy('Date')
df = df.withColumn('sequential_num',row_number().over(windowspec))
df.show()


+-------+----------+--------------+
|GroupID|      Date|sequential_num|
+-------+----------+--------------+
|      A|2023-01-01|             1|
|      A|2023-01-02|             2|
|      B|2023-01-01|             1|
|      B|2023-01-03|             2|
+-------+----------+--------------+



In [ ]:
from pyspark.sql.functions import *
sales_data = [("1", 100), ("2", 150), ("3", None), ("4", 200), ("5", None)]
schema = ["sales_id","amount"]
df = spark.createDataFrame(sales_data,schema)
# avg_here = df.dropna(subset=['amount']).agg(avg('amount').alias('avg_amt')).first()[0]
avg_here = df.na.drop().agg(avg("amount").alias('avg_amt')).first()[0]
print(avg_here)
# avg_here.show()
# all = df.fillna(avg_here)
# all.show()


150.0


In [6]:
#PIVOT the dataframe
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
data = [("Product1", 100, 150, 200),
        ("Product2", 200, 250, 300),
        ("Product3", 300, 350, 400)]

# Columns: Product, Sales_Jan, Sales_Feb, Sales_Mar
columns = ["Product", "Sales_Jan", "Sales_Feb", "Sales_Mar"]
spark = SparkSession.builder.appName("pivoting_table").getOrCreate()
df = spark.createDataFrame(data,columns)
df.show()

pivoted_table = df.selectExpr('Product',"stack(3,'jan',Sales_Jan,'Feb',Sales_Feb,'Mar',Sales_Mar) as (Month,Sales)")
pivoted_table.show()

+--------+---------+---------+---------+
| Product|Sales_Jan|Sales_Feb|Sales_Mar|
+--------+---------+---------+---------+
|Product1|      100|      150|      200|
|Product2|      200|      250|      300|
|Product3|      300|      350|      400|
+--------+---------+---------+---------+

+--------+-----+-----+
| Product|Month|Sales|
+--------+-----+-----+
|Product1|  jan|  100|
|Product1|  Feb|  150|
|Product1|  Mar|  200|
|Product2|  jan|  200|
|Product2|  Feb|  250|
|Product2|  Mar|  300|
|Product3|  jan|  300|
|Product3|  Feb|  350|
|Product3|  Mar|  400|
+--------+-----+-----+

